In [0]:
!pip install dill
!pip install nltk==3.4
!pip install epitran

     |████████████████████████████████| 1.4MB 2.9MB/s 
  Created wheel for nltk: filename=nltk-3.4-cp36-none-any.whl size=1436384 sha256=116b395a80868f4a11e4d905a62f9660ca474676559b6f1bca9bd54203d5921b
  Stored in directory: /root/.cache/pip/wheels/4b/c8/24/b2343664bcceb7147efeb21c0b23703a05b23fcfeaceaa2a1e
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 133kB 2.6MB/s 
     |████████████████████████████████| 276kB 7.5MB/s 
     |████████████████████████████████| 71kB 6.5MB/s 
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-cp36-none-any.whl size=10768 sha256=c06766a5d366f125082c415b25417ad89805663b114701d4b96f904fcefa8d11
  Stored in directory: /root/.cache/pip/wheels/a6/09/e9/e800279c98a0a8c94543f3de6c8a562f60e51363ed26e71283
  Created wheel for marisa-trie: filename=marisa_trie-0.7.5-cp36-cp36m-linux_x86_64.whl size=862307 sha256=bb5ff0d3978bac02

In [0]:
import nltk
nltk.download("punkt")

from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk import word_tokenize, sent_tokenize 
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize.treebank import TreebankWordDetokenizer
detokenize = TreebankWordDetokenizer().detokenize
from nltk.lm import Laplace

import epitran

import itertools

import pandas as pd
import numpy as np
import math
import pickle

import random
from bisect import bisect

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
booba = pd.read_csv("booba.csv",index_col=0)
damso = pd.read_csv("damso.csv",index_col=0)
guizmo = pd.read_csv("guizmo.csv",index_col=0)
kaaris = pd.read_csv("kaaris.csv",index_col=0)
lomepal = pd.read_csv("lomepal.csv",index_col=0)
nekfeu = pd.read_csv("nekfeu.csv",index_col=0)
nepal = pd.read_csv("nepal_clean.csv")
orelsan = pd.read_csv("orelsan.csv",index_col=0)
pnl = pd.read_csv("pnl.csv",index_col=0)
sch = pd.read_csv("sch.csv",index_col=0)
vald = pd.read_csv("vald.csv",index_col=0)
jul = pd.read_csv("jul.csv", index_col=0)


df = nepal
#jul.reset_index(drop=True)
#booba.append(damso).append(guizmo).append(kaaris).append(lomepal).append(nekfeu).append(nepal).append(orelsan).append(pnl).append(sch).append(vald).reset_index(drop=True)

In [0]:
df.head()

,sentences
0,tous mes gavanavas sont pravêts à scuiter
1,décoctions en mélangeant cool-al et nectar fruité
2,jai des taches sur mon jogging les autres sont...
3,jme suis mis une vélocirapta gros jétais pas prêt
4,jarrive sur ton davos comme une pluie acide


In [0]:
vald.head()

,title,lyrics,lyrics_clean
0,3 heures de députés des pu,Des putes et des putes (x5684),des putes et des putes x5684 \n
1,3 heures de surprise,"Suce ma bite, suce ma bite, suce ma bite, suce...",suce ma bite suce ma bite suce ma bite suce...
2,Acacia,"Vitamine verte, pédé, des fois j'm'imagine esp...",vitamine verte pédé des fois j m imagine esp...
3,Accordé,"Ah ouais, ouais, ouais, ouais, ouais, ouais, o...",ah ouais ouais ouais ouais ouais ouais o...
4,Acte de foi,"On ne prône pas la violence, le matos le fait\...",on ne prône pas la violence le matos le fait\...


In [0]:
def initmodel(n=4,df=df):
  splilist=[]
  for lyr in df.sentences:
    lyr.replace("’"," ")
    lyr.replace("“"," ")
    lyr = lyr.split()
    lyr.insert(0,"<s>")
    lyr.append("</s>")
    splilist.append(lyr)

  train_data_clean = (everygrams(sent, 1, n) for sent in splilist)
  sents = flatten(splilist)

  model = Laplace(n)
  model.fit(train_data_clean, sents)
  return model

def _random_generator(seed_or_generator):
    if isinstance(seed_or_generator, random.Random):
        return seed_or_generator
    return random.Random(seed_or_generator)

def _weighted_choice(population, weights, random_generator=None):
    if not population:
        raise ValueError("Can't choose from empty population")
    if len(population) != len(weights):
        raise ValueError("The number of weights does not match the population")
    cum_weights = list(itertools.accumulate(weights))
    total = cum_weights[-1]
    threshold = random_generator.random()
    return population[bisect(cum_weights, total * threshold)]

def generate(model, num_words=1, text_seed=None, random_seed=None):
        text_seed = [] if text_seed is None else list(text_seed)
        random_generator = _random_generator(random_seed)
        if num_words == 1:
            context = (
                text_seed[-model.order + 1 :]
                if len(text_seed) >= model.order
                else text_seed
            )
            samples = model.context_counts(model.vocab.lookup(context))
            while context and not samples:
                context = context[1:] if len(context) > 1 else []
                samples = model.context_counts(model.vocab.lookup(context))
            samples = sorted(samples)
            return _weighted_choice(samples, tuple(model.score(w, context) for w in samples), random_generator), samples
        generated = []
        for _ in range(num_words):
            genword, samples = generate(model=model,num_words=1,text_seed=text_seed + generated,random_seed=random_generator)
            try:
              while genword == generated[-1] and genword == generated[-2]:
                genword = random.choice(list(model.vocab))
            except IndexError:
                pass
            generated.append(genword)
        return generated

def generate_sent(model, num_words, text_seed=None, random_seed=None):
    if text_seed is not None:
      content = [text_seed]
    else:
      content = []
    for token in generate(model=model, num_words=num_words, text_seed=text_seed, random_seed=random_seed):
      if token == "</s>":
        break
      content.append(token)
    return " ".join(content)

def dotproduct(v1, v2):
  return sum((a*b) for a, b in zip(v1, v2))

def length(v):
  return math.sqrt(dotproduct(v, v))

def angle(v1, v2):
  return math.acos(dotproduct(v1, v2) / (length(v1) * length(v2)))

def matrixer(sequence):
  vowels = ["j","w","ɥ","a","ɑ","e","ɛ","ɛː","ə","i","œ","ø","o","ɔ","u","y","ɑ̃","ɛ̃","œ̃","ɔ̃"]
  vector = list()

  epi = epitran.Epitran('fra-Latn')

  sequence = sequence.lower()
  sequence = epi.transliterate(sequence)

  for vowel in vowels:
    vector.append(sequence.count(vowel))

  seq = ''.join([l for l in sequence if l in vowels])
  for vowel in vowels:
    if seq[-1] == vowel:
      vector.append(1)
    else:
      vector.append(0)
  return vector

def compare(context,sequences):
  sequences.insert(0,context)
  vectors = list()
  angles = list()
  
  for seq in sequences:
    vectors.append(matrixer(seq))
  
  for i in range(len(vectors)-1):
    angles.append(angle(vectors[0],vectors[i+1]))

  return sequences[angles.index(min(angles))+1]

def save_model(model,name="model.pickle"):
  f = open(name, 'wb')
  pickle.dump(model, f)
  f.close()

def load_model(model,name="model.pickle"):
  f = open(name, 'rb')
  model = pickle.load(f)
  f.close()
  return model

def couplet(model=initmodel(),sents=5,comp=5):
  sentlist = [generate_sent(model,np.random.randint(5,21))]
  for i in range(sents):
    context = sentlist[-1]
    trials=list()
    for j in range(comp):
      trials.append(generate_sent(model,np.random.randint(5,21)))
      print("Trial",j,"done")
    sentlist.append(compare(context, trials))
    print("Sentence",i,"done")
  return "\n".join(sentlist)

def refrain(model=initmodel(), comp=5, form=None):
  sentlist = [" ".join(model.generate(np.random.randint(5,15)))]
  context = sentlist[0]
  trials = list()
  for j in range(comp):
    trials.append(" ".join(model.generate(np.random.randint(5,15))))
  sentlist.append(compare(context,trials))
  if form == None:
    form = np.random.randint(0,3)
  if form == 0:
    return "\n".join([sentlist[0],sentlist[1],sentlist[0],sentlist[1]])
  elif form == 1:
    return "\n".join([sentlist[0],sentlist[0],sentlist[1],sentlist[1]])
  elif form == 2:
    return "\n".join([sentlist[0],sentlist[1],sentlist[1],sentlist[0]])

In [0]:
print(couplet(initmodel(3,pnl)))

In [0]:
print(refrain(initmodel(3,nepal)))

des séries dsquats quest-ce que
des séries dsquats quest-ce que
àl faute de time </s> dun vieux président mort en 50 exemplaires
àl faute de time </s> dun vieux président mort en 50 exemplaires


In [0]:
print(couplet(initmodel(1,nepal)))

Trial 0 done
Trial 1 done
Trial 2 done
Trial 3 done
Trial 4 done
Sentence 0 done
Trial 0 done
Trial 1 done
Trial 2 done
Trial 3 done
Trial 4 done
Sentence 1 done
Trial 0 done
Trial 1 done
Trial 2 done
Trial 3 done
Trial 4 done
Sentence 2 done
Trial 0 done
Trial 1 done
Trial 2 done
Trial 3 done
Trial 4 done
Sentence 3 done
Trial 0 done
Trial 1 done
Trial 2 done
Trial 3 done
Trial 4 done
Sentence 4 done
j mh qui en les cuiter penchants dièses si
me plus faire à à san quelqu bateaux ulcères gros il
t comme j track un hey la lisa magnifique
cause monde la ra on la que mon going le depuis terrible
gorge refaire bave cristallise toaster t ça 2 alentours l spécial
je même l sugar ça pour pondre quitter croyable remplir
